In [ ]:
from astroquery.simbad import Simbad

# List all fields available for querying in Simbad
table = Simbad.list_votable_fields()
print(table)


--NOTES--

1. The parameter filtername must correspond to an existing filter. Filters include: B,V,R,I,J,K.  They are checked by SIMBAD but not astroquery.simbad

2. Fields beginning with rvz display the data as it is in the database. Fields beginning with rv force the display as a radial velocity. Fields beginning with z force the display as a redshift

3. For each measurement catalog, the VOTable contains all fields of the first measurement. When applicable, the first measurement is the mean one. 

Available VOTABLE fields:

bibcodelist(y1-y2)
biblio
cel
cl.g
coo(opt)
coo_bibcode
coo_err_angle
coo_err_maja
coo_err_mina
coo_qual
coo_wavelength
coordinates
dec(opt)
dec_prec
diameter
dim
dim_angle
dim_bibcode
dim_incl
dim_majaxis
dim_minaxis
dim_qual
dim_wavelength
dimensions
distance
distance_result
einstein
fe_h
flux(filtername)
flux_bibcode(filtername)
flux_error(filtername)
flux_name(filtername)
flux_qual(filtername)
flux_system(filtername)
flux_unit(filtername)
fluxdata(filtername)
gcrv
gen
gj
hbet
hbet1
hgam
id(opt)
ids
iras
irc
iso
iue
jp11
link_bibcode
main_id
measurements
membership
mesplx
mespm
mk
morphtype
mt
mt_bibcode
mt_qual
otype
otype(opt)
otypes
parallax
plx
plx_bibcode
plx_error
plx_prec
plx_qual
pm
pm_bibcode
pm_err_angle
pm_err_maja
pm_err_mina
pm_qual
pmdec
pmdec_prec
pmra
pmra_prec
pos
posa
propermotions
ra(opt)
ra_prec
rot
rv_value
rvz_bibcode
rvz_error
rvz_qual
rvz_radvel
rvz_type
rvz_wavelength
sao
sp
sp_bibcode
sp_nature
sp_qual
sptype
td1
typed_id
ubv
uvby
uvby1
v*
velocity
xmm
z_value


For more information on a field:
Simbad.get_field_description ('field_name') 

Currently active VOTABLE fields:

 ['main_id', 'coordinates', 'z_value', 'flux(B)', 'z_value', 'flux(B)', 'z_value', 'flux(B)', 'z_value', 'flux(B)', 'z_value', 'flux(B)', 'z_value', 'flux(B)', 'z_value', 'flux(B)', 'z_value', 'flux(B)', 'z_value', 'flux(B)', 'z_value', 'flux(B)', 'z_value', 'flux(B)', 'z_value', 'flux(B)', 'z_value', 'flux(B)', 'z_value', 'flux(B)']
None

In [ ]:
Simbad.list_columns()['column_name'][0:200]

In [ ]:
# Access astronomical databases
from pyvo import registry  # version >=1.6

# Moc and HEALPix tools
from mocpy import MOC

# Coordinates manipulation
from astropy.coordinates import SkyCoord

# Sky visualization
from ipyaladin import Aladin  # version >=0.4.0

# For plots
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from astroquery.simbad import Simbad
from astropy.cosmology import FlatLambdaCDM
import astropy.units as uu
import astropy.constants as cc
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['figure.dpi'] = 100 # 200 e.g. is really fine, but slower
plt.rcParams['font.size'] = 20 


# the catalogue name in VizieR
CATALOGUE = "J/MNRAS/390/466"
# each resource in the VO has an identifier, called ivoid. For vizier catalogs,
# the VO ids can be constructed like this:
catalogue_ivoid = f"ivo://CDS.VizieR/{CATALOGUE}"
# the actual query to the registry
voresource = registry.search(ivoid=catalogue_ivoid)[0]

tables = voresource.get_tables()
print(f"In this catalogue, we have {len(tables)} tables.")
for table_name, table in tables.items():
    print(f"{table_name}: {table.description}")
tables_names = list(tables.keys())


# get the first table of the catalogue
first_table_name = tables_names[0]

# execute a synchronous ADQL query
tap_service = voresource.get_service("tap")
tap_records = tap_service.search(
    f'select * from "{first_table_name}"',
)
tap_records

In [ ]:
df = tap_records.to_table().to_pandas()
df.columns, df.shape

# ugc 763
V(km/s) 1151.4 [~] / z(emission) 0.003848 [~] / cz 1153.62 [~]
   (Rad) C 2004MNRAS.350.1195M
Morphological type:	SAB C 2019MNRAS.488..590B
Angular size (arcmin):	1.397 0.810 135 (NIR) C 2006AJ....131.1163S
Fluxes (11) :	
B 12.1 [~] D 2005MNRAS.361...34D
R 12.0 [~] D 2005MNRAS.361...34D
I 12.1 [~] D 2005MNRAS.361...34D
J 10.665 [0.014] C 2006AJ....131.1163S
H 9.654 [0.061] C 2006AJ....131.1163S
K 9.385 [0.065] C 2006AJ....131.1163S
u (AB) 14.429 [0.008] C 2009ApJS..182..543A
g (AB) 13.136 [0.002] C 2009ApJS..182..543A
r (AB) 12.502 [0.002] C 2009ApJS..182..543A
i (AB) 12.218 [0.002] C 2009ApJS..182..543A
z (AB) 12.106 [0.003] C 2009ApJS..182..543A


In [ ]:

# Constants
zero_point = 48.6 # Adjust based on calibration
c= cc.c.to(uu.km/uu.s).value
R0 = (14.01E9*uu.lyr).to(uu.Mpc).value


def calcBMAG(z, flux, distance):
    # Convert flux to apparent magnitude (assuming flux is in Janskys and using B-band as example)
    apparent_mag = -2.5 * np.log10(flux / 3631)
    
    # Calculate distance modulus
    distance_modulus = 5 * np.log10(distance.to(uu.pc).value / 10)
    
    # Calculate absolute magnitude
    b_mag = apparent_mag - distance_modulus
    # print(f"BMag: {b_mag}")
    return b_mag

    
def calculate_z_cmb(row):
    if pd.notna(row['z_helio']):
        return helio_to_cmb(row['z_helio'], row['RAJ2000'], row['DEJ2000'])
    return np.nan

def hms_to_deg(ra_hms):
    hours, minutes, seconds = map(float, ra_hms.split())
    return (hours + minutes/60 + seconds/3600) * 15
    
def helio_to_cmb(z_helio, ra, dec):
    # Constants
    c = 299792.458  # speed of light in km/s
    v_sun_cmb = 370  # Sun's velocity relative to CMB in km/s
    lambda_cmb = 168.0  # RA in degrees
    beta_cmb = -7.0  # Dec in degrees
    
    # Convert angles from degrees to radians
    lambda_cmb_rad = np.radians(lambda_cmb)
    beta_cmb_rad = np.radians(beta_cmb)
    ra_rad = np.radians(ra)
    dec_rad = np.radians(dec)
    
    # Calculate the projection of Sun's motion on the line of sight to the object
    cos_theta = (np.sin(beta_cmb_rad) * np.sin(dec_rad) +
                 np.cos(beta_cmb_rad) * np.cos(dec_rad) * np.cos(ra_rad - lambda_cmb_rad))
    v_los = v_sun_cmb * cos_theta  # line of sight component of the Sun's motion

    # Convert heliocentric redshift to CMB redshift
    z_cmb = (z_helio * c + v_los) / c

    return z_cmb

# Add the 'z_value' field to the query to get the redshift
Simbad.add_votable_fields('z_value')
Simbad.add_votable_fields('flux(B)')  # Adds B-band magnitudes
Simbad.add_votable_fields('distance') # adds Distance
df["z_helio"] = np.nan
df["z_cmb"] = np.nan
df["one_plus_z"] = np.nan
df['Simbad_Distance'] = np.nan
df['Simbad_flux'] = np.nan
df['Dist_CMB_z'] = np.nan
df['Vmax'] = df['Vmax'].astype('float64')

# Iterate over the DataFrame
for index, row in df.iterrows():
    obj = Simbad.query_object(row['Name'])
    if obj is not None and len(obj) > 0:
        df.at[index, 'z_helio'] = obj['Z_VALUE'][0] if obj['Z_VALUE'][0] else np.nan
        df.at[index, 'Simbad_flux'] = obj['FLUX_B'][0] if obj['FLUX_B'][0] else np.nan
        df.at[index, 'Simbad_Distance'] =  obj['Distance_distance'][0] if obj['Distance_distance'][0] else np.nan
 
# Assuming 'z_helio', 'RAJ2000', and 'DEJ2000' are already populated correctly
df['z_cmb'] = df.apply(calculate_z_cmb, axis=1)
# Assuming 'Dist' needs to be converted to Mega Parsecs and used along with 'z_helio' and 'Flux'
df["BMAG_calc_helio_z_Flux"] = [calcBMAG(z, flux, dist * uu.Mpc) for z, flux, dist in zip(df.z_helio, df.Flux, df.Dist)]
df['one_plus_z'] = 1 + df.z_cmb
df['Dist_CMB_z'] = R0*df.z_cmb/(1+df.z_cmb)
df['H0'] = df.Vsys/df.Dist


M_sun = 5.48  # Absolute magnitude of the sun in B-band
# Calculate mass from BMAG_CMB
df['Mass'] = 10 ** (-0.4 * (df['BMAG'] - M_sun))
# Calculate Luminosity from BMAG_CMB
df['L'] = 10 ** (-0.4 * (df['BMAG'] - M_sun))

# Remove rows where 'Vmax' is NaN
# df = df.dropna(subset=['Vmax', "BMAG" ])
# df = df[df['Vmax'] > 0]

df.to_excel("./Ghasp_Data/df_GHasp_Simbad.xlsx")

In [ ]:
df

In [ ]:
df = df[[ 'UGC', 'Name', 'Dist', 'BMAG', 'Vmax', 'Vsys', 'NGC', 'one_plus_z',
         'Simbad_Distance', 'Simbad_flux', 'Mass', 'L','H0',
       'RAJ2000', 'DEJ2000', 'Scale', 'Flux', 'z_helio', 'z_cmb', 'Dist_CMB_z']]
df = df.dropna(subset=['Vmax', "BMAG", 'Dist', 'z_cmb' ])
df = df[df['H0'] > 0]
df = df[df['H0'] > 60]
df = df[df['H0'] < 90]
df.shape

In [ ]:
import pandas as pd
import numpy as np

# Assuming df is your DataFrame and H0 is a column in df
# Calculate the median or mean H0 if it's not supposed to be row-wise
# current_H0 = df['H0'].median()  # or .mean(), depending on the dataset

# New H0 value you want to apply
new_H0 = 69.69

# Scaling factor for distances based on the change in H0
df['scale_factor'] = df['H0'] / new_H0

# Check for any potentially invalid scale factors
if (df['scale_factor'] <= 0).any():
    raise ValueError("Scale factor must be positive and non-zero.")

# Update Distances and Vmax
df['Dist'] *= df['scale_factor']
df['Vmax'] *= df['scale_factor']

# Update BMAG based on distance changes
df['BMAG_H0_Corrected'] = df['BMAG'] + 5 * np.log10(df['scale_factor'])

# Recalculate H0 based on new distances and Vsys
df['H0_corrected'] = df['Vsys'] / df['Dist']

# Optionally drop the temporary scale factor column if no longer needed
# df.drop(columns='scale_factor', inplace=True)


# Display updated DataFrame
print(df.head())


In [ ]:
import numpy as np
import pandas as pd
import astropy.units as uu
# Example of fitting a regression model using scikit-learn
from sklearn.linear_model import LinearRegression
from astropy.cosmology import FlatLambdaCDM
import matplotlib.pyplot as plt


# Convert DataFrame columns to numpy arrays and handle NaNs
X = df['BMAG_H0_Corrected'].to_numpy().reshape(-1, 1)
Y = df['BMAG'].to_numpy().reshape(-1, 1)

# Check and handle NaN values before fitting the model
mask = ~np.isnan(X).any(axis=1) & ~np.isnan(Y).any(axis=1)
X = X[mask]
Y = Y[mask]

# Linear regression model
linear_regressor = LinearRegression()
linear_regressor.fit(X, Y)  # Perform linear regression
Y_pred = linear_regressor.predict(X)  # Make predictions

# Output the coefficients
alpha = linear_regressor.coef_[0][0]
beta = linear_regressor.intercept_[0]

# Plotting
plt.figure(figsize=(10, 6))
plt.scatter(X, Y, c='blue', label='Data Points')
plt.plot(X, Y_pred, color='red', label='Fit: Y={:.2f}X + {:.2f}'.format(alpha, beta))
plt.xlabel('BMAG_H0_Corrected')
plt.ylabel('BMAG')
plt.title('BMAG vs BMAG_H0_Corrected')
plt.gca().invert_yaxis()  # Invert y-axis as brighter objects have lower magnitude
plt.legend()
plt.grid(True)
plt.show()

# print(f"Coefficient (alpha): {alpha}, Intercept (beta): {beta}")


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression


X = np.log10(df['Vmax'].to_numpy().reshape(-1, 1))
# Calculate apparent magnitude from Flux
Y = df['BMAG'].to_numpy().reshape(-1, 1)

# Ensure that 'X' and 'Y' are not containing NaN values
if np.isnan(X).any() or np.isnan(Y).any():
    print("Warning: NaN values detected in X or Y, check your data preprocessing steps.")

# Linear regression model
linear_regressor = LinearRegression()
linear_regressor.fit(X, Y)  # Perform linear regression
Y_pred = linear_regressor.predict(X)  # Make predictions

# Output the coefficients
alpha = linear_regressor.coef_[0][0]
beta = linear_regressor.intercept_[0]

# Plotting
plt.figure(figsize=(10, 6))
plt.scatter(X, Y, c='blue', label='Data Points')
plt.plot(X, Y_pred, color='red', label='Fit: Y={:.2f}X + {:.2f}'.format(alpha, beta))
plt.xlabel('LogVmax (km/s)')
plt.ylabel('BMAG')
plt.title('Tully-Fisher Relation')
plt.gca().invert_yaxis()  # Invert y-axis as brighter objects have lower magnitude
plt.legend()
plt.grid(True)
plt.savefig("./Drawing_For_Publications/Tully_Fisher_GHasp.png")
plt.show()

# print(f"Coefficient (alpha): {alpha}, Intercept (beta): {beta}")


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression


X = np.log10(df['Vmax'].to_numpy().reshape(-1, 1))
# Calculate apparent magnitude from Flux
Y = df['BMAG_H0_Corrected'].to_numpy().reshape(-1, 1)

# Ensure that 'X' and 'Y' are not containing NaN values
if np.isnan(X).any() or np.isnan(Y).any():
    print("Warning: NaN values detected in X or Y, check your data preprocessing steps.")

# Linear regression model
linear_regressor = LinearRegression()
linear_regressor.fit(X, Y)  # Perform linear regression
Y_pred = linear_regressor.predict(X)  # Make predictions

# Output the coefficients
alpha = linear_regressor.coef_[0][0]
beta = linear_regressor.intercept_[0]

# Plotting
plt.figure(figsize=(10, 6))
plt.scatter(X, Y, c='blue', label='Data Points')
plt.plot(X, Y_pred, color='red', label='Fit: Y={:.2f}X + {:.2f}'.format(alpha, beta))
plt.xlabel('LogVmax (km/s)')
plt.ylabel('BMAG_H0_Corrected')
plt.title('Tully-Fisher Relation')
plt.gca().invert_yaxis()  # Invert y-axis as brighter objects have lower magnitude
plt.legend()
plt.grid(True)
plt.show()

print(f"Coefficient (alpha): {alpha}, Intercept (beta): {beta}")


Torres-Flores, S., Epinat, B., Amram, P., Plana, H., & Mendes de Oliveira, C. (2011). GHASP: An Hα kinematic survey of spiral and irregular galaxies - IX. The near-infrared, stellar and baryonic Tully-Fisher relations. Monthly Notices of the Royal Astronomical Society, 416(3), 1936–1948. https://doi.org/10.1111/J.1365-2966.2011.19169.X